# Warm up
Let's start with some warm up exercises to get you familiar with the database and do some SQL querying.
Make sure to have downloaded the NBA dataset form Kaggle as described in the README.

Firstly, the purpose of the __name__ == "__main__" construct is to determine whether a Python script is being run as the main program or if it is being imported as a module in another script. When a Python file is executed, the special built-in variable __name__ is set to "__main__". However, if the file is imported as a module, __name__ is set to the module's name. This enables us to run things from the command line.

In [ ]:
import sqlite3 as sql
import pandas as pd
from pathlib import Path

## TODO: Use pathlib to get the path to the data directory. Path(__file__) won't work in a Jupyter notebook.
DATA_PATH = Path.cwd() / 'data'

## Create a SQLite connection and run some SQL queries

In [ ]:
con = sql.connect("/home/cab252/ps_2_ex3_database/data/nba.sqlite")  # connect to the database
query = 'SELECT * FROM game LIMIT 10'  # write a query that selects the first 10 rows from the game table
top_10_game = pd.read_sql(query, con)  # read the query into a pandas dataframe
top_10_game.head()  # print the dataframe

In [ ]:
# let's look at the schema of the database
# TODO: sqlite_master is the name of the table you want to inspeact: Replace TABLE_NAME with the name of the table you want to inspect

query = """
    SELECT *
    FROM sqlite_master
"""

pd.read_sql(query, con).sort_values('name')


In [ ]:
# let's see all the tables we have in the database
# TODO: Filter for tables only
query = """
    SELECT name
    FROM sqlite_master
    WHERE type = 'table'
    ORDER BY name;
"""
pd.read_sql(query, con)

In [ ]:
# let's have a look at the columns in the game table
query = """
    PRAGMA table_info(draft_combine_stats)
"""
pd.read_sql(query, con)

In [ ]:
# let's find the earliest recorded game
query = """
    SELECT game_date
    FROM game
    ORDER BY game_date
    LIMIT 1
"""

print(pd.read_sql(query, con))

# let's find the latest recorded game
query = """
    SELECT game_date
    FROM game
    ORDER BY game_date DESC
    LIMIT 1
"""
print(pd.read_sql(query, con))

In [ ]:
query = """
    SELECT 
        ROUND(100.0 * SUM(CASE WHEN wl_home = 'W' THEN 1 ELSE 0 END) / COUNT(*), 2) AS home_win_pct,
        ROUND(100.0 * SUM(CASE WHEN wl_away = 'W' THEN 1 ELSE 0 END) / COUNT(*), 2) AS away_win_pct
    FROM game;
"""
win_rates = pd.read_sql(query, con)
print(win_rates)


In [ ]:
query = """
    SELECT
        ROUND(AVG(pts_home), 2) AS avg_home_points,
        ROUND(AVG(pts_away), 2) AS avg_away_points
    FROM game;
"""
avg_points = pd.read_sql(query, con)
print(avg_points)


In [ ]:
query = """
    SELECT
        team_name_home AS team_name,
        ROUND(AVG(pts_home), 2) AS avg_home_points,
        ROUND(AVG(pts_away), 2) AS avg_away_points,
        ROUND(AVG(pts_home) - AVG(pts_away), 2) AS home_advantage
    FROM game
    GROUP BY team_name_home
    ORDER BY home_advantage DESC;
"""
team_comparison = pd.read_sql(query, con)
print(team_comparison.head(10))


In [ ]:
import matplotlib.pyplot as plt

# Plot home vs away win rates
plt.bar(['Home', 'Away'], [win_rates['home_win_pct'][0], win_rates['away_win_pct'][0]], color=['#1f77b4', '#ff7f0e'])
plt.title('Home vs Away Win Percentage')
plt.ylabel('Win %')
plt.ylim(0, 100)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

# Plot team-level advantage
team_comparison.head(10).plot(
    x='team_name',
    y='home_advantage',
    kind='bar',
    figsize=(8,5),
    color='#2ca02c',
    legend=False
)
plt.title('Top 10 Teams with Strongest Home-Court Advantage')
plt.ylabel('Average Points Difference (Home - Away)')
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()


It is clear that there is a home advantage, which is particularly strong for certain teams.